### Setup

In [3]:
# install if not yet done
using Pkg

# Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
# Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

    Updating registry at `C:\Users\domin\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
    Updating git-repo `https://github.com/mppmu/BoostFractor.jl.git`
  No Changes to `C:\Users\domin\OneDrive\Desktop\Julia Files\Dragoon.jl\Project.toml`
  No Changes to `C:\Users\domin\OneDrive\Desktop\Julia Files\Dragoon.jl\Manifest.toml`


In [4]:
using Dragoon

In [ ]:
n = 20 # number of discs in the booster

initdist = findpeak1d(20.025e9,n) # initial disc configuration

freqs = genFreqs(20.025e9,50e6; n=10) # optimize on these frequencies
freqsplot = genFreqs(20.025e9,150e6; n=1000) # plot with these frequencies

booster = AnalyticalBooster(initdist; ndisk=n)  # physical properties of the booster

hist = initHist(booster,2*(booster.ndisk^2),freqs,objective); # stores measurements
#                           ^ length, should be at least n

### Optimizer

In [ ]:
trace = nelderMead(booster,hist,freqs,
                    1.,1+2/n,0.75-1/2n,1-1/n,1e-9,
                    ObjAnalytical,
                    InitSimplexRegular(1e-4),
                    DefaultSimplexSampler,
                    UnstuckDont;
                    maxiter=Int(1e3),
                    showtrace=true,
                    showevery=100,
                    unstuckisiter=true,)

In [ ]:
# obtain result, booster is now in optimized position
pos2dist(booster.pos)     #in distance space
# booster.pos               #in position space

In [ ]:
# get handy output
plt = analyse(hist,trace,freqsplot; freqs=freqs,div=10)

# savefig(plt[i],"cool_epic_result_wow.svg")

In [ ]:
booster.timestamp